In [1]:
%load_ext lab_black
import pandas as pd
import numpy as np
from os.path import join
import matplotlib.pyplot as plt
import scipy.stats
import statsmodels.api as sm

In [2]:
prefix = "hsq-0.25-pcausal-0.01-hermodel-gcta"
sim_i = 3

In [3]:
test_indiv_list = pd.read_csv(
    "/u/project/sgss/UKBB/PRS-RESEARCH/00-compile-data/out/INDIVLIST/eur_test.indiv",
    sep="\t",
    header=None,
    names=["FID", "IID"],
)
test_indiv_list = (
    test_indiv_list.FID.astype(str) + "_" + test_indiv_list.IID.astype(str)
).values

In [4]:
df_score = pd.read_csv(
    join(f"out/PRS-SCORE/{prefix}", f"sim_{sim_i}.score_summary.tsv.gz"),
    sep="\t",
    index_col=0,
).reindex(test_indiv_list)

In [5]:
SIM_DIR = f"../01-simulate-pheno/out/PHENO/{prefix}"

# read genetic value and phenotype
df_pheno_g = (
    pd.read_csv(
        join(f"{SIM_DIR}/sim.pheno_g.tsv.gz"),
        sep="\t",
        index_col=0,
    )[[f"SIM_{sim_i}"]]
    .rename(columns={f"SIM_{sim_i}": "GV"})
    .reindex(df_score.index)
)
df_pheno = (
    pd.read_csv(join(f"{SIM_DIR}/sim.pheno.tsv.gz"), sep="\t", index_col=0)[
        [f"SIM_{sim_i}"]
    ]
    .rename(columns={f"SIM_{sim_i}": "PHENO"})
    .reindex(df_score.index)
)
df_score = pd.concat([df_pheno, df_pheno_g, df_score], axis=1)

In [6]:
intercept = (df_score.MEAN - df_score.GV).mean()
df_plot = pd.DataFrame(
    {
        "GV": df_score.GV,
        "MEAN": df_score.MEAN - intercept,
        "LOWER": df_score.QUANTILE_10 - intercept,
        "UPPER": df_score.QUANTILE_90 - intercept,
    }
)

In [7]:
scipy.stats.pearsonr(df_plot.GV, df_plot.MEAN)

(0.7769047327602104, 0.0)

In [8]:
np.mean(df_plot.GV <= df_plot.LOWER)

0.14746392140271705

In [9]:
np.mean(df_plot.GV <= df_plot.UPPER)

0.8543693884266441

In [10]:
sm.OLS(df_plot.GV, sm.add_constant(df_plot.MEAN)).fit().summary()

/u/project/pasaniuc/kangchen/software/miniconda3/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     GV   R-squared:                       0.604
Model:                            OLS   Adj. R-squared:                  0.604
Method:                 Least Squares   F-statistic:                 3.239e+04
Date:                Tue, 21 Dec 2021   Prob (F-statistic):               0.00
Time:                        14:31:29   Log-Likelihood:                -5439.5
No. Observations:               21273   AIC:                         1.088e+04
Df Residuals:                   21271   BIC:                         1.090e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0091      0.003     -3.128      0.002      -0.015      -0.003
MEAN           1.0264      0.006    179.964      0.000       1.015       1.038
==============================================================================
Omnibus:                       46.334   Durbin-Watson:                   1.978
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               53.445
Skew:                          -0.060   Prob(JB):                     2.48e-12
Kurtosis:                       3.215   Cond. No.                         3.02
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""